#### importing packages

In [1]:
import pandas as pd
# import matplotlib.pyplot as plt

from carriageway import Carriageway
from impact_factor import impact
from load import ll_A, ll_70R, ll_70RT
from reaction import find_bm, find_sf, find_ra, find_rb
from bridge_specs import box,bearing

In [2]:
span=box.span
c_c=bearing.c_c

# live load calculations

#### defining load (vehicles)
load pair contains pair (wheel position, wheel load)

In [3]:
vehicles = [ll_A, ll_70R, ll_70RT]
classA_pair, class70R, class70RT = [list(i.loadpair) for i in vehicles]
loads = [classA_pair, class70R, class70RT]

#### maxBM, maxSF at equal intervals
when each vehicle from the list of vehicles travels along the span, max reactions (BM and SF+/-) at equally divided intervals are calculated and stored in maxBMs, maxSFs_plus, maxSFs_minus.

In [4]:


maxBMs = []
maxSFs_plus = []
maxSFs_minus = []
for i in range(len(loads)):
    maxBM = []
    maxSF_plus = []
    maxSF_minus = []
    for j in range(9):
        at = span / 8 * j
        first_wheel_at = 0
        step = 0.1

        BM = find_bm(span, 0, at)
        SF_plus = find_sf(span, 0, at)
        SF_minus = find_sf(span, 0, at)
        for k in range(int((span + loads[i][-1][0]) / step) + 2):  # '+2' to make sure the load moves all the way to
            # end until it has no effect
            bm = 0
            sf = 0
            for this in loads[i]:
                a, load = this
                pos = -a + first_wheel_at
                bm = bm + find_bm(span, pos, at) * load
                sf = sf + find_sf(span, pos, at) * load
            first_wheel_at += step
            BM = bm if bm > BM else BM
            SF_plus = sf if sf > SF_plus else SF_plus
            SF_minus = sf if sf < SF_minus else SF_minus
        maxBM.append(round(BM, 3))
        maxSF_plus.append(round(SF_plus, 3))
        maxSF_minus.append(round(SF_minus, 3))
    maxBMs.append(maxBM)
    maxSFs_plus.append(maxSF_plus)
    maxSFs_minus.append(maxSF_minus)

#### make a dataframe that contains maxBMs, maxSFs_plus, maxSFs_minus

In [5]:
A = ['MaxBM', 'MaxSF+', 'MaxSF-']
B = ['ClassA', 'Class70RW', 'Class70RT']

# A = ['ClassA', 'Class70RW', 'Class70RT']
# B = ['MaxBM', 'MaxSF+', 'MaxSF-']

iterables = [A, B]
index = pd.MultiIndex.from_product(iterables)

C = []

for i in [maxBMs, maxSFs_plus, maxSFs_minus]:
    C.extend(i)

# for i in range(len(loads)):
#     for j in [maxBMs, maxSFs_plus, maxSFs_minus]:
#         C.append(j[i])

df = pd.DataFrame(C, index=index, columns=[span / 8 * i for i in range(9)])
# print(df.loc[('ClassA', 'MaxSF-')])   ## you can navigate using loc, iloc

new_row = df.loc['MaxSF+'].where(df.loc['MaxSF+'] > abs(df.loc['MaxSF-']), abs(df.loc['MaxSF-']))

new_row.index = pd.MultiIndex.from_product([['MaxSF'], B])
df = pd.concat([df, new_row])

# df.to_excel('outputs/loads.xlsx') # uncomment to save as excel file

*how to read Excel file*

In [6]:
# df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])

# ###get index names###
# A = df.index.get_level_values(0).drop_duplicates().to_list()
# B = df.index.get_level_values(1).drop_duplicates().to_list()
# print(A, B)

#### impact factor

In [7]:
IF = [impact(i.name, span) for i in vehicles]
IF

[0.088, 0.088, 0.088]

#### possible combinations for given carriageway width

In [8]:
cwWidth = 6
carriageway = Carriageway(cwWidth)
combinations = carriageway.combinations()
combination_val = [i.get_value() for i in combinations]

#### eccentricity calculation
a combination consists of multiple arrangements gives different eccentricities. maximum eccentricities for each combination calculated.

In [9]:
list_combinations = []
list_max_e = []
list_plot = []
for i in combinations:
    list_combinations.append([i.classA, i.class70Rw, i.class70Rt])
    arrangements = i.arrangements()
    list_arrangements = []
    eccentricity = []
    max_e = 0
    for k in arrangements:
        if k.check_exceedance() * k.check_from_right() == 1:
            list_arrangements.append(k.veh)
            e = k.eccentricity()
            eccentricity.append(e)
            max_e = e if abs(e) > abs(max_e) else max_e
    list_max_e.append(max_e)

list_max_e

[-3.739698609263685e-16, -0.40500000000000025, -0.3500000000000001]

#### make a dataframe for combinations and their corresponding eccentricities

In [10]:
df1 = pd.DataFrame(list_combinations, columns=['ClassA', 'Class70Rw', 'Class70Rt'],
                  index=[f'comb{i + 1}' for i in range(len(list_combinations))])
df1['MaxEccentricity'] = list_max_e

*export eccentricity as excel file (uncomment)*

In [11]:
df1.to_excel('outputs/max_e.xlsx')

PermissionError: [Errno 13] Permission denied: 'outputs/max_e.xlsx'

#### total reactions for given combination
total reactions = sum of (maxreactions at mid span * impact factor * no. of that vehicles)

In [ ]:
totalBMs = [(df.loc['MaxBM'][span/2]*IF).dot(combination_val[i]) for i in range(len(combinations))]
totalSFs = [(df.loc['MaxSF'][span/2]*IF).dot(combination_val[i]) for i in range(len(combinations))]
combination_val, totalBMs, totalSFs

In [ ]:
df

In [ ]:
df1['TotalBM'] = totalBMs
df1['TotalSF'] = totalSFs

In [ ]:
df1

# loads on peir

In [ ]:
import math

In [ ]:
abs(math.log10(step))

In [ ]:


reactions = []

max_sums = []
lefts = []
rights = []
max_sums_at = []
for veh in vehicles:
    dummy_load = list(veh.loadpair)
    no = 2
    nose_dist = 20
    if veh.name == 'Class 70RT':
        nose_dist = 90
        no = 1
    elif veh.name == 'Class 70RW':
        nose_dist = 31.52
        
    final_load = list(dummy_load)
    max_sum_at = 0
    for i in range(no-1):
        delta = dummy_load[-1][0] + nose_dist
        final_load.extend([(i[0] + delta, i[1]) for i in dummy_load])
        
    sumlr = 0
    nleft = 0
    nright = 0
    
    head_at = 0
    step = 0.01
    for i in range(int(90/step)):
        left = 0
        right = 0
        for j in final_load:
            rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*j[1]
            ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*j[1]
            temp_load = 0
            if head_at-j[0] == span+c_c/2:
#                 print('hjgjh')
                temp_load = (j[1])/2
                ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*temp_load
                rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*temp_load
            left+=rb
            right+=ra
        
        if left+right>sumlr:
            sumlr = left+right
            nleft, nright = left, right
            max_sum_at = head_at
            
        head_at= round(head_at + step, int(abs(math.log10(step))))
    max_sums.append(sumlr)
    lefts.append(nleft)
    rights.append(nright)
    max_sums_at.append(max_sum_at)

In [ ]:
df1=pd.DataFrame([lefts, rights, max_sums], columns=['class A', '70RW', '70RT'], index=['RL', "RR", 'Sum']).T



In [ ]:
df1.to_csv('outputs/Live_Loads_Seismic.csv')


In [ ]:
find_ra(30, -0.75, 0.75, 0.75)

*check*

In [ ]:
veh = ll_A
dummy_load = list(veh.loadpair)
no = 2
nose_dist = 20
if veh.name == 'Class 70RT':
    nose_dist = 90
    no = 1
elif veh.name == 'Class 70RW':
    nose_dist = 31.52

final_load = list(dummy_load)
for i in range(no-1):
    delta = dummy_load[-1][0] + nose_dist
    final_load.extend([(i[0] + delta, i[1]) for i in dummy_load])
    
left = []
right = []
head_at = 75.05

for j in final_load:
    rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*j[1]
    ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*j[1]
    temp_load = 0
    if head_at-j[0] == span+c_c/2:
#         print('hjgjh')
        temp_load = (j[1])/2
        ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*temp_load
        rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*temp_load
    left.append(rb)
    right.append(ra)

In [ ]:
pd.DataFrame({'pos' : [head_at - i[0] for i in final_load], 'load' : [i[1] for i in final_load], 'left' : left, 'right' : right} )

In [ ]:
sum(left), sum(right), sum(left)+sum(right)